In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import os
import netCDF4
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import catboost
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
summary_dir = '/Trex/test_case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/summary'
# summary_dir = '/Users/yguo/DataSpellProjects/hw/uhi'


# merged_feather_path = os.path.join(summary_dir, 'local_hour_adjusted_variables.feather')
merged_feather_path = os.path.join(summary_dir, 'local_hour_adjusted_variables_with_location_ID_event_ID.feather')

local_hour_adjusted_df = pd.read_feather(merged_feather_path)
local_hour_adjusted_df.info()

In [ ]:
location_ID_path = os.path.join(summary_dir, 'location_IDs.nc')
location_ID_ds = xr.open_dataset(location_ID_path, engine='netcdf4')

In [ ]:
location_ID_ds

In [ ]:
df_daily_vars = pd.read_excel('../Data/hourlyDataSchema.xlsx')
daily_vars =df_daily_vars.loc[df_daily_vars['X_vars2'] == 'Y', 'Variable']
daily_var_lst = daily_vars.tolist()
daily_var_lst

In [ ]:
df_daily_vars.info()

In [ ]:
# Step 1: Define masks for daytime and nighttime
daytime_mask = local_hour_adjusted_df['local_hour'].between(8, 16)
nighttime_mask = (local_hour_adjusted_df['local_hour'].between(20, 24) |
                  local_hour_adjusted_df['local_hour'].between(0, 4))


In [ ]:
daytime_uhi_diff = local_hour_adjusted_df[nighttime_mask]
# nighttime_uhi_diff = local_hour_adjusted_df[nighttime_mask]

X = daytime_uhi_diff[daily_var_lst]
y = daytime_uhi_diff['UHI_diff']

In [ ]:
X.info()
y.info()

#  feature selection

In [ ]:
import catboost
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_pool = Pool(X_train, y_train)
validation_pool = Pool(X_val, y_val)

#  Running the feature selection algorithm

#   SHAP waterfall plot

#  Functions to create model and graph

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
import shap

def train_and_evaluate(local_hour_adjusted_df, time_mask, df_daily_vars):
    # Filter dataframe for daytime data
    time_uhi_diff = local_hour_adjusted_df[time_mask]
    daily_vars =df_daily_vars.loc[df_daily_vars['X_vars2'] == 'Y', 'Variable']
    daily_var_lst = daily_vars.tolist()
    # Select features and target
    X = time_uhi_diff[daily_var_lst]
    y = time_uhi_diff['UHI_diff']

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    train_pool = Pool(X_train, y_train)
    validation_pool = Pool(X_val, y_val)

    # Train the final model with all columns on selected features after feature selection
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=5,
        loss_function='RMSE',
        eval_metric='RMSE',
        random_seed=42,
        task_type='GPU',
        verbose=False
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val),
                       use_best_model=True, early_stopping_rounds=50, plot=True)
    return model

def importance_plot(model, validation_pool, df_daily_vars):
    # Get long_feature_names
    feature_names = validation_pool.get_feature_names()
    feature_name_mapping = pd.Series(df_daily_vars['Long Name'].values, index=df_daily_vars['Variable']).to_dict()
    long_feature_names = [feature_name_mapping.get(name, "Unknown Feature") for name in feature_names]
    # Calculate SHAP values    
    explainer = shap.TreeExplainer(model, feature_names=long_feature_names)
    shap_values = explainer.shap_values(validation_pool)
    mean_abs_shap_values = np.abs(shap_values).mean(axis=0)
    # Normalize the SHAP values so that they sum to one
    total = mean_abs_shap_values.sum()
    normalized_shap_values = mean_abs_shap_values / total
    # Plot the SHAP values for the first instance in the validation dataset
    shap.waterfall_plot(shap.Explanation(normalized_shap_values, base_values=explainer.expected_value,
                                         data=validation_pool.get_features()[0],
                                         feature_names=validation_pool.get_feature_names()))
    
    shap.plots.waterfall(shap_values)


In [ ]:
night_model = train_and_evaluate(local_hour_adjusted_df, nighttime_mask, df_daily_vars=df_daily_vars)

In [ ]:
day_model = train_and_evaluate(local_hour_adjusted_df, daytime_mask, df_daily_vars=df_daily_vars)

In [ ]:
# Get feature importance
day_feature_importances = day_model.get_feature_importance(type='FeatureImportance')
day_feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': day_feature_importances})
print(day_feature_importance_df.sort_values(by='Importance', ascending=False))


In [ ]:
from catboost import EFstrType

# Get feature importance
night_feature_importances = night_model.get_feature_importance(type='FeatureImportance')
night_feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': night_feature_importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))


In [ ]:
day_feature_importance_df.sum()

In [ ]:
import catboost
from catboost import *
import shap
shap.initjs() 

In [ ]:
shap_values = day_model.get_feature_importance(Pool(X, y), type='ShapValues')

expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]

# visualize the first prediction's explanation
shap.force_plot(expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
shap_values[0,:]

In [ ]:
shap_values

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X)

In [ ]:
shap.dependence_plot("U10", shap_values, X)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(X['U10'], y)
plt.title('Plot of ColumnX vs ColumnY')
plt.xlabel('ColumnX')
plt.ylabel('ColumnY')
plt.grid(True)
plt.show()